<a href="https://colab.research.google.com/github/KarishmaSaxena/naturalLanguageProcessing/blob/main/sentimentanalysis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import nltk

In [ ]:
dataset = pd.read_csv('/content/Reviews.csv', nrows=250000)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset['Text'][0]

EDA

In [ ]:
dataset['Score'].value_counts().sort_index().plot(kind='bar',title='count of reviews by stars',xlabel='count',ylabel='score',figsize=(10,5))

**basic** **nltk**

In [ ]:
example = dataset['Text'][0]

In [ ]:
example

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
word = nltk.word_tokenize(example)
print(word)

In [ ]:
sentences = nltk.sent_tokenize(example)
sentences

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
ps.stem(example)

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatize = WordNetLemmatizer()

In [ ]:
lemmatize.lemmatize(example)

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')
word = word_tokenize(example)
pos_tags = nltk.pos_tag(word)

In [ ]:
pos_tags

In [ ]:
nltk.download('maxent_ne_chunker_tab')
nltk.download('corpora/words.zip/words/')
nltk.download('words')
named_entities = nltk.ne_chunk(pos_tags)

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('chunkers/maxent_ne_chunker_tab/english_ace_multiclass/')
from nltk import word_tokenize, pos_tag, ne_chunk

In [ ]:
!pip install svgling
named_entities

VADER = Valence Aware Dictionary and sEntiment Reasoner


A rule-based sentiment analysis tool

Designed especially for social media & reviews

 Works out of the box (no training needed)


 ADER gives 4 scores for a sentence:

{
 'neg': 0.0,  negative

 'neu': 0.54, neutral

 'pos': 0.46, postive

 'compound': 0.6249 Compound
}

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm import tqdm

sentimnet_analyzer = SentimentIntensityAnalyzer()

In [ ]:
sentimnet_analyzer

In [ ]:
sentimnet_analyzer.polarity_scores(example)

In [ ]:
sentimnet_analyzer.polarity_scores("I am so Happy")

In [ ]:
sentimnet_analyzer.polarity_scores("I am so sad")

In [ ]:
example[49:]

In [ ]:
sentimnet_analyzer.polarity_scores(example[49:])

In [ ]:
data = pd.DataFrame(np.array(dataset['Text']), columns=['Text'])

In [ ]:
data

In [ ]:
df=dataset.head(500)

In [ ]:
res = {}
for index, row in tqdm(df.iterrows(), total = len(df)):
   text = row['Text']
   myid = row['Id']
   res[myid] = sentimnet_analyzer.polarity_scores(text)


In [ ]:
res

In [ ]:
# in df form ploarity score
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index':'Id'})
vaders = vaders.merge(df, how='left')

In [ ]:
# merged ploarity score into orginal dataset
vaders

In [ ]:
vaders.head()

In [ ]:
sns.barplot(data=vaders, x='Score', y='compound')
plt.xlabel('Score')
plt.ylabel('compound')
plt.title('compound score by review score')

In [ ]:
fig_size,axs = plt.subplots(1,3,figsize=(15,5))
sns.barplot(data=vaders, x='Score', y='pos',ax=axs[0], color='blue')
sns.barplot(data=vaders, x='Score', y='neg',ax=axs[1], color='red')
sns.barplot(data=vaders, x='Score', y='neu',ax=axs[2], color='green')

axs[0].set_title('positive')
axs[1].set_title('negative')
axs[2].set_title('neutral')
plt.show()


**RoBERTa Pretrained Model - (Robustly Optimized BERT Approach)**

A Transformer-based language model

Learns context from surrounding words

Uses:

Self-attention

Bidirectional context

Large-scale pretraining

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = "cardiffnlp/roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# vader example on example
print(example)
sentimnet_analyzer.polarity_scores(example)

In [ ]:
# run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scroes = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}

In [ ]:
print(scores_dict)

In [ ]:
def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
  }
  return scores_dict

In [ ]:
from tqdm import tqdm
res = {}
for i ,row in tqdm(df.iterrows(), total=len(df)):
  try:
      text = row['Text']
      myid = row['Id']
      vaders_result = sentimnet_analyzer.polarity_scores(text)
      roberta_result = polarity_scores_roberta(text)

      both_merged = {**vaders_result, **roberta_result}
      res[myid] = both_merged

  except RuntimeError:
      print(f'Broke for id {myid}')

In [ ]:
vaders_result

In [ ]:
roberta_result

In [ ]:
both_merged = {**vaders_result, **roberta_result}
res[myid] = both_merged

In [ ]:
both_merged

In [ ]:
results_df =pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index':'Id'})
results_df = results_df.merge(df, how='left')

In [ ]:
results_df.head()

# compare scores btw models

In [ ]:
results_df.columns

In [ ]:
sns.pairplot(data=results_df, vars=['neg','neu','pos',
                                   'roberta_neg' , 'roberta_neu' , 'roberta_pos'],
             hue='Score', palette='tab10')
plt.show()

Review Examples:

In [ ]:
results_df.query('Score == 1').sort_values('roberta_pos', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 1').sort_values('pos' , ascending=False)['Text'].values[0]

# negative sentimnet 5 star review

In [ ]:
results_df.query('Score == 5').sort_values('roberta_neg', ascending=False)['Text'].values[0]

In [ ]:
results_df.query('Score == 5').sort_values('neg',ascending=False)['Text'].values[0]

In [ ]:
from transformers import pipeline
sent_pipeline = pipeline("sentiment-analysis")

In [ ]:
sent_pipeline('I am so happy')

In [ ]:
sent_pipeline('I am so sad')

In [ ]:
sent_pipeline('At morining something good happend i thought my day will good also but now i sad ')